wikipedia情報からキャラクターの情報を構造化してまとめたjsonファイルを作成するソースコード

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia

wikipedia.set_lang("ja")
res = wikipedia.search("ベートーベン")

wp = wikipedia.page(res[0])
wp_content = wp.content

In [ ]:
import re

def extract_dictionary(text):
    # パターン: '== キー ==' の形式を見つける
    pattern = r'={2,3}\s*(.*?)\s*={2,3}'
    
    # キーの位置を見つける
    keys = re.findall(pattern, text)
    
    # キーの位置を基に、次のキーの直前までをバリューとして抽出
    positions = [match.span() for match in re.finditer(pattern, text)]
    values = [text[positions[i][1]:positions[i + 1][0] if i + 1 < len(positions) else None].strip() for i in range(len(keys))]

    if (len(values) == 0 or values[0] == ""):
        return ""
    
    # print(values)

    # 辞書に格納
    # return values
    return dict(zip(keys, values))



In [ ]:
import re

def extract_hierarchy(texts):
    hierarchy = {}
    current_parent = None
    current_subsection = None

    for text in texts:
        parent_match = re.search(r'^==\s(.*?)\s==', text)
        subsec_match = re.search(r'^===\s(.*?)\s===', text)

        if parent_match:
            # 親コンテンツを処理
            current_parent = parent_match.group(1)
            current_subsection = None
            parent_content = text[parent_match.end()+1:] if parent_match.end() is not None else ''
            
            hierarchy[current_parent] = {'content': parent_content, 'subsections': {}}
        elif subsec_match and current_parent:
            # サブコンテンツを処理
            current_subsection = subsec_match.group(1)
            subsection_content = text[subsec_match.end()+1:] if subsec_match.end() is not None else ''
            hierarchy[current_parent]['subsections'][current_subsection] = subsection_content

    return hierarchy


In [ ]:
wp_content_list = wp_content.split("\n\n\n")

wp_content_list_after_scrutiny = []

for content in wp_content_list:
    wp_content_list_after_scrutiny.append(content)

content_dict = extract_hierarchy(wp_content_list_after_scrutiny)


In [ ]:
import json

with open("wiki.json","w") as f:
    json.dump(content_dict,f,ensure_ascii=False)